In [15]:
from itertools import product
from typing import Tuple, Literal, Callable

import numpy as np
import scipy as sp
import numpy.linalg as linalg
import scipy.linalg
import pandas as pd
import plotly.graph_objects as go

from enum import Enum

# Util

In [16]:
matrix_size_name = "matrix_size"
matrix_a_condition_name = "cond(A)"
matrix_l_condition_name = "cond(L)"
matrix_u_condition_name = "cond(U)"
error_column_name = "lu_error"
qr_error_column_name = "qr_error"
alpha_column_name = "alpha"
regulirized_matrix_a_condition_name = "cond(A + alpha * E)"
regulirized_matrix_l_condition_name = "cond(reg(L))"
regulirized_matrix_u_condition_name = "cond(reg(U))"
regulirized_error_column_name = "reg_error"

class ConditionNumberType(Enum):
    SpectralCondition = "cond_s"
    VolumCondition = "cond_v"
    AngleCondition = "cond_a"


class SolutionType(Enum):
    LUDecomposition = "lu"
    QRRotationDecomposition = "qr_rotation"
    QRReflectionDecomposition = "qr_reflection"

In [17]:
def calculate_spectral_condition_number(matrix):
    return linalg.cond(matrix)


def calculate_volume_condition_number(matrix):
    volume_denominator = abs(linalg.det(matrix))

    volume_numerator = 1
    for row in matrix:
        volume_numerator *= linalg.norm(row)

    return volume_numerator / volume_denominator


def calculate_angle_condition_number(matrix):
    candidates = []
    for row, column in zip(matrix, linalg.inv(matrix).T):
        candidates.append(linalg.norm(row) * linalg.norm(column))

    return max(candidates)


def calculate_condition_number(condition_number_type: ConditionNumberType, matrix):
    return {
        ConditionNumberType.AngleCondition: calculate_angle_condition_number,
        ConditionNumberType.SpectralCondition: calculate_spectral_condition_number,
        ConditionNumberType.VolumCondition: calculate_volume_condition_number,
    }[condition_number_type](matrix)

In [18]:
def generate_matrix(element_factory, size):
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ]
    )

In [19]:
def draw_compare_plot_for_size_and_cond(data, matrix_condition_names=None):
    if matrix_condition_names is None:
        matrix_condition_names = [matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name]

    fig = go.Figure()

    for matrix_condition_name in matrix_condition_names:
        fig.add_scatter(x=data[matrix_size_name], y=data[matrix_condition_name], name=matrix_condition_name)

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Число обусловленности', tickformat='.2e')
    fig.update_layout(title='Зависимость числа обусловленности от размера матрицы')

    fig.show()

In [58]:
def draw_compare_plot_for_size_and_error(datasets, error_columns):
    fig = go.Figure()

    if not isinstance(datasets, list):
        datasets = [datasets]

    if not isinstance(error_columns, list):
        error_columns = [error_columns]

    assert len(error_columns) == len(datasets)

    for dataset, error_column in zip(datasets, error_columns):
        fig.add_scatter(x=dataset[matrix_size_name], y=dataset[error_column], name=error_column)

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title='Зависимость ошибки от размера матрицы')

    fig.show()

In [21]:
def get_lup_decomposition(matrix):
    return sp.linalg.lu(matrix)

def solve_using_lu_decompostion(l, u, right_part):
    y = sp.linalg.solve_triangular(l, right_part, lower=True)
    return sp.linalg.solve_triangular(u, y, lower=False)

In [37]:
def get_rotation_matrix(dim, i, j, cos, sin):
    matrix = np.eye(dim)

    matrix[i, j] = -sin
    matrix[j, i] = sin

    matrix[i, i] = cos
    matrix[j, j] = cos

    return matrix


def get_qr_rotation_decomposition(matrix):
    dim = matrix.shape[0]

    Q = np.eye(dim)
    R = matrix.copy()

    for i in range(dim - 1):
        for j in range(i + 1, dim):
            z_i = R[i, i]
            z_j = R[j, i]

            if np.isclose(z_j, 0):
                continue

            r = np.hypot(z_i, z_j)

            cos = z_i / r
            sin = -z_j / r
            T = get_rotation_matrix(dim, i, j, cos, sin)

            Q = T.dot(Q)
            R = T.dot(R)

    return Q.T, R

In [23]:
def get_qr_reflection_decomposition(matrix):
    dim = matrix.shape[0]

    q = np.eye(dim)
    r = matrix.copy()

    for i in range(dim):
        x = r[:, i].copy()
        x[:i] = np.zeros(i)

        alpha = linalg.norm(x) if x[i] > 0 else -linalg.norm(x)

        e = np.zeros(dim)
        e[i] = 1

        u = x + alpha * e
        v = u / linalg.norm(u)

        current_q = np.eye(dim) - 2 * np.outer(v, v)

        q = q.dot(current_q)
        r = current_q.dot(r)

    return q, r

In [24]:
def solve_using_qr_decomposition(q, r, right_part):
    return sp.linalg.solve_triangular(r, q.T.dot(right_part), lower=False)

In [25]:
def run_benchmark(
        matrix,
        right_part,
        expected_solution,
        condition_number_type: ConditionNumberType = ConditionNumberType.AngleCondition,
        solution_type: SolutionType = SolutionType.LUDecomposition
):
    decompose = {
        SolutionType.LUDecomposition: get_lup_decomposition,
        SolutionType.QRReflectionDecomposition: get_qr_reflection_decomposition,
        SolutionType.QRRotationDecomposition: get_qr_rotation_decomposition,
    }

    if solution_type == SolutionType.LUDecomposition:
        p, first, second = get_lup_decomposition(matrix)
    else:
        first, second = decompose[solution_type](matrix)

    matrix_condition_number = calculate_condition_number(condition_number_type, matrix)
    first_condition_number = calculate_condition_number(condition_number_type, first)
    second_condition_number = calculate_condition_number(condition_number_type, second)

    if solution_type == solution_type.LUDecomposition:
        actual_solution = solve_using_lu_decompostion(first, second, (np.linalg.inv(p)).dot(right_part))
    else:
        actual_solution = solve_using_qr_decomposition(first, second, right_part)

    return pd.Series(
        [
            matrix_condition_number,
            first_condition_number,
            second_condition_number,
            linalg.norm(expected_solution - actual_solution),
        ]
    )

# Хорошие примеры из Пакулиной

In [26]:
def get_pakulina_1_linear_system():
    matrix = np.array([
        [3.278164, 1.046583, -1.378574],
        [1.046583, 2.975937, 0.934251],
        [-1.378574, 0.934251, 4.836173],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [28]:
def get_pakulina_6_linear_system():
    matrix = np.array([
        [9.016024, 1.082197, 2.783575],
        [1.08219, 6.846595, 0.647647],
        [-2.78357, 0.647647, 5.432541],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [29]:
def get_pakulina_11_linear_system():
    matrix = np.array([
        [6.687233, 0.80267, 2.06459],
        [0.8026, 5.07816, 0.48037],
        [-2.06459, 0.48037, 4.02934],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [30]:
good_data = pd.DataFrame.from_dict({
    1: run_benchmark(*get_pakulina_1_linear_system()),
    6: run_benchmark(*get_pakulina_6_linear_system()),
    11: run_benchmark(*get_pakulina_11_linear_system()),
}, orient='index')
good_data.columns = [matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name, error_column_name]
good_data

,cond(A),cond(L),cond(U),lu_error
1,1.906892,1.203120,1.270739,7.447602e-16
6,1.065856,1.058710,1.053435,3.330669e-16
11,1.065853,1.058708,1.053435,2.220446e-16


In [31]:
solution_type = SolutionType.QRReflectionDecomposition
good_data = pd.DataFrame.from_dict({
    1: run_benchmark(*get_pakulina_1_linear_system(), solution_type=solution_type),
    6: run_benchmark(*get_pakulina_6_linear_system(), solution_type=solution_type),
    11: run_benchmark(*get_pakulina_11_linear_system(), solution_type=solution_type),
}, orient='index')
good_data.columns = [matrix_a_condition_name, "cond(Q)", "cond(R)", qr_error_column_name]
good_data

,cond(A),cond(Q),cond(R),qr_error
1,1.906892,1.0,1.894137,6.661338e-16
6,1.065856,1.0,1.034388,6.280370e-16
11,1.065853,1.0,1.034387,3.140185e-16


In [38]:
solution_type = SolutionType.QRRotationDecomposition
good_data = pd.DataFrame.from_dict({
    1: run_benchmark(*get_pakulina_1_linear_system(), solution_type=solution_type),
    6: run_benchmark(*get_pakulina_6_linear_system(), solution_type=solution_type),
    11: run_benchmark(*get_pakulina_11_linear_system(), solution_type=solution_type),
}, orient='index')
good_data.columns = [matrix_a_condition_name, "cond(Q)", "cond(R)", qr_error_column_name]
good_data

,cond(A),cond(Q),cond(R),qr_error
1,1.906892,1.0,1.894137,4.577567e-16
6,1.065856,1.0,1.034388,4.965068e-16
11,1.065853,1.0,1.034387,3.330669e-16


# Плохие примеры (матрица Гильберта)

In [49]:
def get_hilbert_linear_system(size, expected_solution=None):
    matrix = np.around(generate_matrix(lambda row, column: 1 / (row + column - 1), size), 10)
    
    expected_solution = expected_solution if expected_solution is not None else np.ones(size)
    right_part = np.dot(matrix, expected_solution)

    return matrix, right_part, expected_solution

In [50]:
lu_hilbert_data = pd.DataFrame(range(2, 100), columns=[matrix_size_name])
lu_hilbert_data[
    [matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name, error_column_name]
] = lu_hilbert_data.apply(
    lambda row: run_benchmark(*get_hilbert_linear_system(row[matrix_size_name])),
    axis=1,
)
lu_hilbert_data.sort_values(by=error_column_name, ascending=False)

,matrix_size,cond(A),cond(L),cond(U),lu_error
69,71,2.727659e+13,11.851698,9.396397e+05,3.134550e-02
80,82,1.297828e+11,13.062564,1.203102e+06,5.896167e-04
82,84,3.416746e+11,12.410363,1.260514e+06,5.302875e-04
26,28,1.860718e+11,6.516365,3.425235e+04,4.357577e-04
25,27,1.731782e+11,6.228150,3.036746e+04,3.500657e-04
...,...,...,...,...,...
4,6,2.441337e+06,2.388348,7.774785e+00,8.558862e-10
3,5,9.515739e+04,2.438588,4.304053e+00,1.507071e-13
1,3,1.728872e+02,1.500000,1.634693e+00,1.389554e-14
2,4,4.020914e+03,2.173067,2.660409e+00,9.452556e-15


In [52]:
qr_rotate_hilbert_data = pd.DataFrame(range(2, 100), columns=[matrix_size_name])
qr_rotate_hilbert_data[
    [matrix_a_condition_name, "cond(q)", "cond(r)", "qr_rotate_error"]
] = qr_rotate_hilbert_data.apply(
    lambda row: run_benchmark(*get_hilbert_linear_system(row[matrix_size_name]), solution_type=SolutionType.QRRotationDecomposition),
    axis=1,
)
qr_rotate_hilbert_data.sort_values(by="qr_rotate_error", ascending=False)

,matrix_size,cond(A),cond(q),cond(r),qr_rotate_error
97,99,6.349683e+09,1.0,8.357166e+06,4.795930e+03
96,98,7.069957e+09,1.0,7.901752e+06,4.657596e+03
95,97,9.308828e+10,1.0,4.866125e+07,4.285934e+03
81,83,3.574778e+11,1.0,3.065952e+08,3.160005e+03
83,85,1.888802e+10,1.0,1.344227e+07,3.043017e+03
...,...,...,...,...,...
4,6,2.441337e+06,1.0,7.494749e+00,6.454775e-11
3,5,9.515739e+04,1.0,4.486153e+00,3.642108e-11
2,4,4.020914e+03,1.0,2.697358e+00,5.998854e-13
1,3,1.728872e+02,1.0,1.643704e+00,5.925929e-15


In [53]:
qr_reflect_hilbert_data = pd.DataFrame(range(2, 100), columns=[matrix_size_name])
qr_reflect_hilbert_data[
    [matrix_a_condition_name, "cond(q)", "cond(r)", "qr_reflect_error"]
] = qr_reflect_hilbert_data.apply(
    lambda row: run_benchmark(*get_hilbert_linear_system(row[matrix_size_name]), solution_type=SolutionType.QRReflectionDecomposition),
    axis=1,
)
qr_reflect_hilbert_data.sort_values(by="qr_reflect_error", ascending=False)

,matrix_size,cond(A),cond(q),cond(r),qr_reflect_error
69,71,2.727659e+13,1.0,8.397454e+05,3.265125e-01
82,84,3.416746e+11,1.0,1.440483e+06,3.540437e-02
81,83,3.574778e+11,1.0,1.362096e+06,1.776514e-02
56,58,1.939232e+11,1.0,4.658738e+05,4.689131e-03
95,97,9.308828e+10,1.0,2.161456e+06,4.018761e-03
...,...,...,...,...,...
4,6,2.441337e+06,1.0,7.494749e+00,1.726090e-09
3,5,9.515739e+04,1.0,4.486153e+00,5.660460e-12
2,4,4.020914e+03,1.0,2.697358e+00,4.121533e-13
1,3,1.728872e+02,1.0,1.643704e+00,4.401163e-14


In [59]:
draw_compare_plot_for_size_and_error(
    [lu_hilbert_data, qr_reflect_hilbert_data, qr_rotate_hilbert_data], 
    [error_column_name, "qr_reflect_error", "qr_rotate_error"],
)